# Implementación experimental
## Integrantes:  

* Anthony Daniel Bautista León
* Cristopher Sebastián García Pacheco
* Rosa Irene Limachi Paucar  
* Julio Rosales Joaquin  
  
  
**Fecha de entrega: 9-noviembre-2018**

Empezamos con el uso de manejo de archivos en python para crear una base de datos en la que guardaremos y vericaremos los datos de ciertos usuarios. Haciendo uso de la librería *csv* podemos manejar este tipo de archivos.

In [1]:
import csv

def crear_archivo(base_de_datos):
    archivo = open(base_de_datos, "w")
    archivo.write("Usuario,Contraseña\n")
    archivo.close()

def guardar_datos(base_de_datos, datos):	#base_de_datos es un archivo con los datos, datos es una lista
    archivo = open(base_de_datos, "a")		#abrimos el archivo para escritura al final
    archivo_csv = csv.writer(archivo)
    archivo_csv.writerows(datos)			#llenamos la fila del archivo
    archivo.close()							#cerramos el archivo

def existencia_usuario(base_de_datos, usuario):
    ok = 0
    archivo = open(base_de_datos, "r")
    archivo_csv = csv.reader(archivo)
    for row in archivo_csv:				#vemos linea por linea
        if row[0] == usuario:			#si existe el usuario
            ok = 1
            break
    archivo.close()
    if ok == 0:
        return False
    elif ok == 1:
        return True
    
def validar_datos(base_de_datos, datos):
    ok = 0
    archivo = open(base_de_datos, "r")
    archivo_csv = csv.reader(archivo)
    for row in archivo_csv:								#vemos linea por linea
        if row[0] == datos[0] and row[1] == datos[1]:	#si coincide el usuario y contraseña
            ok = 1
            break
    archivo.close()
    if ok == 0:
        return False
    elif ok == 1:
        return True

Ahora implementamos el Hash del cuco para encriptar la entrada de datos, en este caso, lo usaremos para encriptar contraseña, de esta forma solo la tendrá el usuario.

In [2]:
n = 7			#tamaño de las tablas hash
ver = 2			#numero de tablas y funciones hash
hashTable = [[i*0 for i in range(n)], [i*0 for i in range(n)]]	#inicializamos las tablas en 0
pos = [0, 0]	#guardara las salidas de la funcion hash
aux = []		#guardara los elementos que no pueda posicionar por encontrar un ciclo.

def funcion_hash(funcion, clave):
    suma = 0
    for i in clave:
        suma+=ord(i)			#suma de ascii de cada letra
    if(funcion == 1):
        return suma%n
    elif(funcion == 2):
        return n-(suma%n)-1
        

def colocar(clave, tabla, cont, n): 
    if (cont == n):								#detecta el bucle al intentar posicionar
        aux.append(clave)						#guardamos el elemento en una lista auxiliar
        return
    for i in range(ver):
        pos[i] = funcion_hash(i+1, clave)		#hallamos los indices para posicionarlo en cada tabla
    if (hashTable[tabla][pos[tabla]] != 0):		#si encuentra un elemento en la posicion que quiero
        save = hashTable[tabla][pos[tabla]]		#guardamos ese elemento
        hashTable[tabla][pos[tabla]] = clave	#y posicionamos la clave en el lugar que queremos
        colocar(save, (tabla+1)%ver, cont+1, n)	#colocando el que estaba ahi, en la otra tabla
    else:
        hashTable[tabla][pos[tabla]] = clave;	#si no hay un elemento, simplemente lo coloca ahi

def actualizar_tabla():
    global hashTable
    hashTable = [[i*0 for i in range(n)], [i*0 for i in range(n)]]    
        
def cuckoo(valores, n):
    for i in valores:
        cont = 0
        colocar(i, 0, cont, n)				#colocar todos los valores
    contraseña = [l for l in hashTable[0] if l != 0] + [l for l in hashTable[1] if l != 0] + aux #concatenamos sin los ceros
    cadena = "".join(contraseña)			#lo convertimos a cadena
    actualizar_tabla()						#actualizamos la tabla en ceros al final para la siguiente cadena
    return cadena


Como ya tenemos la base del programa, ahora veremos qué hacer con los datos que recibimos, cómo usarlos y dónde guardarlos.

In [3]:
crear_archivo("base_de_datos.csv")		#creamos el archivo para la base de datos


def recibir_para_guardar():
    ex = True
    while ex:
        usuario = input("\nIngrese usuario: ")
        ex = existencia_usuario("base_de_datos.csv", usuario)	#vemos si existe ese nombre de usuario en la base de datos
        if ex:
            print("El nombre de usuario ya existe.")
    print("Nombre de usuario válido.")
    contraseña = input("Ingrese contraseña: ")
    return usuario, contraseña

def recibir_para_validar():
    usuario = input("\nIngrese usuario: ")
    contraseña1 = input("Ingrese contraseña: ")
    contraseña2 = transformar_entrada(contraseña1)
    return usuario, contraseña2

def transformar_entrada(entrada):
    cadena_aux = (",").join(entrada)		#añadimos comas entre cada letra del string	
    cadena = cadena_aux.split(",")			#y lo convertimos a lista de letras
    return cadena


def guardando_datos():
    usuario, contraseña = recibir_para_guardar()
    cadena = cuckoo(contraseña, len(contraseña))		#aplicamos el hash del cuco a la contraseña
    entrada = [(usuario, cadena)]
    guardar_datos("base_de_datos.csv", entrada)		#y lo guardamos(encriptado) en la base de datos
    print("Bienvenido {}, te registraste con éxito.".format(usuario))
    
def validando_datos():
    usuario, contraseña = recibir_para_validar()
    cadena = cuckoo(contraseña, len(contraseña))		#aplicamos el hash del cuco a la contraseña
    entrada = (usuario, cadena)
    ex = validar_datos("base_de_datos.csv", entrada)	#y verificamos si esta en la base de datos
    if ex:
        print("El usuario está registrado.")
    else:
        print("Nombre de usuario o contraseña incorrectos.")
        

Utilizaremos un menú para interactuar con el usuario.

In [4]:
def menu():
    print("\n======================================================")
    print("\t\tAcceso de usuario")
    print("======================================================")
    print("\t1.- Registrarse.")
    print("\t2.- Validar usuario.")
    print("\t3.- Salir.")

def repetir_menu():
    while True:
        menu()
        opcion = input("Ingrese opcion: ")
        if opcion == "1":
            guardando_datos()
        elif opcion == "2":
            validando_datos()
        elif opcion == "3":
            print("Gracias por usar el sistema de login.")
            break
        else:
            print("Pulsa una opcion correcta.")

def main():
    repetir_menu()

Finalmente llamamos a la función principal para empezar el programa

In [5]:
main()


		Acceso de usuario
	1.- Registrarse.
	2.- Validar usuario.
	3.- Salir.
Ingrese opcion: 1

Ingrese usuario: mario
Nombre de usuario válido.
Ingrese contraseña: qwerty
Bienvenido mario, te registraste con éxito.

		Acceso de usuario
	1.- Registrarse.
	2.- Validar usuario.
	3.- Salir.
Ingrese opcion: 1

Ingrese usuario: pepe
Nombre de usuario válido.
Ingrese contraseña: asdfgh
Bienvenido pepe, te registraste con éxito.

		Acceso de usuario
	1.- Registrarse.
	2.- Validar usuario.
	3.- Salir.
Ingrese opcion: 1

Ingrese usuario: pepe
El nombre de usuario ya existe.

Ingrese usuario: anthony
Nombre de usuario válido.
Ingrese contraseña: zxcvb
Bienvenido anthony, te registraste con éxito.

		Acceso de usuario
	1.- Registrarse.
	2.- Validar usuario.
	3.- Salir.
Ingrese opcion: 2

Ingrese usuario: pepe
Ingrese contraseña: asdfgh
El usuario está registrado.

		Acceso de usuario
	1.- Registrarse.
	2.- Validar usuario.
	3.- Salir.
Ingrese opcion: 3
Gracias por usar el sistema de login.


Notaremos que en nuestra carpeta se ha creado un archivo llamado **"base_de_datos.csv"**, este se reescribirá cada vez que iniciemos el programa. Podremos abrir el archivo o verlo de una forma más interesante. Utilizaremos la librería *pandas* para visalizarlo en este cuaderno. 

In [7]:
import pandas as pd

tabla = pd.read_csv("base_de_datos.csv")
print(tabla)

   Usuario Contraseña
0    mario     wqyetr
1     pepe     dsfgha
2  anthony      bczvx


Como vemos, las contraseñas están encriptadas, de esta forma, son un poco más seguras.